In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import utils 
import torch
from torch import nn
from torchsummary import summary
from einops import rearrange, reduce, repeat

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(project="transformer_LSTM_AE")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: acercyc. Use `wandb login --relogin` to force relogin


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:

class SpiralDataset(torch.utils.data.Dataset):
    def __init__(self, nTime=72, nBatch=128, seed=0):
        self.data = utils.SynthData.spiral(nTime, nBatch, seed)
        
    def __getitem__(self, idx):
        return self.data[idx, :, :]
        
    def __len__(self):
        return self.data.shape[0]
    
    
dataset_train = SpiralDataset(nTime=72, nBatch=128, seed=0)
dataset_val = SpiralDataset(nTime=72, nBatch=128, seed=1)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=128, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(dataset_train, batch_size=128, shuffle=True)


In [4]:
class DecoderLayer_LSTM_AE(nn.Module):
    def __init__(self, nFeature=32, dropout=0.1):
        super(DecoderLayer_LSTM_AE, self).__init__()
        self.lstm = nn.LSTM(nFeature, nFeature, 1, batch_first=True)
        self.silu = nn.SiLU()
        self.batchNorm = nn.BatchNorm1d(nFeature)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc=None):
        # b t f
        res = x
        if hc is None:
            x, hidden = self.lstm(x)
        else:
            x, hidden = self.lstm(x, hc)
        x = self.silu(x)
        x = x + res
        x = rearrange(x, 'b t f -> b f t')
        x = self.batchNorm(x)
        x = rearrange(x, 'b f t -> b t f')
        x = self.dropout(x)
        return x


class Decoder_LSTM_AE(nn.Module):
    def __init__(self, nLayer=8, nFeature=32, dropout=0.1):
        super(Decoder_LSTM_AE, self).__init__()
        self.seq = nn.Sequential()
        for i in range(nLayer):
            self.seq.add_module(
                f'lstm{i}', DecoderLayer_LSTM_AE(nFeature=nFeature))

    def forward(self, x):
        x = self.seq(x)
        return x
    
class TrajNet(nn.Module):
    def __init__(self, nHidden=8, nFeature=32, nhead=8, dim_feedforward=32, num_layers=8, dropout=0.1):
        super(TrajNet, self).__init__()
        self.nhidden = nHidden
        self.nFeature = nFeature

        # encoding
        self.conv_enc = nn.Conv1d(3, nFeature, 1)
        encoder_ = nn.TransformerEncoderLayer(d_model=nFeature,
                                              nhead=nhead,
                                              dim_feedforward=dim_feedforward,
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_, num_layers=num_layers)
        self.hidden = nn.Linear(nFeature, nHidden)
        
        # decode
        self.conv_dec1 = nn.Conv1d(nHidden, nFeature, 1)
        self.decoder = nn.LSTM(nFeature, nFeature, 4, batch_first=True)
        # self.decoder = Decoder_LSTM_AE(
        #     nLayer=8, nFeature=nFeature, dropout=dropout)
        # decoder_ = nn.TransformerEncoderLayer(d_model=nFeature, 
        #                                       nhead=nhead, 
        #                                       dim_feedforward=dim_feedforward, 
        #                                       batch_first=True, 
        #                                       activation='gelu',
        #                                       dropout=dropout)
        # self.decoder = nn.TransformerEncoder(decoder_, num_layers=num_layers) 
        # generator_ = nn.TransformerEncoderLayer(d_model=nFeature, 
        #                                         nhead=nhead, 
        #                                         dim_feedforward=dim_feedforward, 
        #                                         batch_first=True, 
        #                                         activation='gelu',
        #                                         dropout=dropout)
        # # generation   
        # self.generator = nn.TransformerEncoder(generator_, num_layers=num_layers)
        self.conv_dec2 = nn.Conv1d(nFeature, 2, 1)
        
    
    @staticmethod
    def positionEncoding(x):
        # x: b t f 
        nBatch = x.shape[0]
        nTime = x.shape[1]
        p = torch.arange(0, nTime).type_as(x)
        p = p / 300
        p = repeat(p, 't -> b t f', b=nBatch, f=1)
        x = torch.concat([x, p], dim=2)
        return x
        
        
    def forward(self, x):
        # x: b t f 
        nBatch = x.shape[0]
        nTime = x.shape[1]
        
        # --------------------------------- encoding --------------------------------- #
        # expend features
        x = self.positionEncoding(x)
        x = rearrange(x, 'b t f -> b f t')
        x = self.conv_enc(x)
        x = rearrange(x, 'b f t -> b t f')
        
        # ----------------------------- hidden bottleneck ---------------------------- #
        x = x[:, 0, :]
        x = self.hidden(x)
        hidden = torch.tanh(x)
        y = torch.tanh(hidden)
        
        # --------------------------------- decoding --------------------------------- #
        y = repeat(y, 'b f -> b t f', t=nTime)
        # y_ = torch.zeros((nBatch, nTime-1, self.nhidden))
        # y = torch.concat([y, y_], dim=1)
        
        y = rearrange(y, 'b t f -> b f t')
        y = self.conv_dec1(y)
        y = rearrange(y, 'b f t -> b t f')
    
        
        y, _ = self.decoder(y)

        y = rearrange(y, 'b t f -> b f t')
        y = self.conv_dec2(y)
        y = rearrange(y, 'b f t -> b t f')

        return y 
        
        
class PL_model(pl.LightningModule):
    def __init__(self):
        super(PL_model, self).__init__()
        self.model = TrajNet()
        self.c = 0
       
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        y = self.forward(batch)
        loss = torch.nn.functional.mse_loss(y, batch)
        self.log('train_loss', loss)        
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
        
    def training_epoch_end(self, training_step_outputs):        
        if self.c % 50 == 0:
            self.eval()
            
            # training set
            x = dataset_train[0:1]
            self.eval()
            y = self.forward(torch.from_numpy(x).double())
            x = x.squeeze()
            y = y.squeeze()
            y = y.detach().numpy()
            ax[0].clear()
            ax[0].plot(x[:, 0], x[:, 1], '-')
            ax[0].plot(y[:, 0], y[:, 1], '-')
            ax[0].plot(0, 0, 'or')
            ax[0].axis('equal')
            
            # evaluation set
            x = dataset_val[0:1]
            y = self.forward(torch.from_numpy(x).double())
            x = x.squeeze()
            y = y.squeeze()
            y = y.detach().numpy()
            ax[1].clear()
            ax[1].plot(x[:, 0], x[:, 1], '-')
            ax[1].plot(y[:, 0], y[:, 1], '-')
            ax[1].plot(0, 0, 'or')
            ax[1].axis('equal')            
                        
            img = utils.Plot.fig2img(fig)
            wandb_logger.log_image('traj', [img])
            self.c = 0
        self.c += 1
          
# %matplotlib qt

# x = torch.randn((3, 4, 2)).type(model.dtype)
fig, ax = plt.subplots(1, 2)
model = PL_model().double()
trainer = pl.Trainer(max_epochs=10000, logger=wandb_logger, log_every_n_steps=10)
trainer.fit(model, dataloader_train)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/acercyc/anaconda3/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(

  | Name  | Type    | Params
----------------------------------
0 | model | TrajNet | 86.2 K
----------------------------------
86.2 K    Trainable params
0         Non-trainable params
86.2 K    Total params
0.345     Total estimated model params size (MB)
/home/acercyc/anaconda3/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine)

Training: 0it [00:00, ?it/s]